# Tutorial 03: Viewing predictions

The process of viewing predictions consists basically on juxtaposing the predictions we created and the raster images. Therefore, we will need to load the rasters we used on the previous tutorials to compute the features and the predictions that we stored. Let's get started.

In [1]:
import pandas as pd
import rasterio
from matplotlib import pyplot as plt
from datetime import date

from damage.features import RasterSplitter

def plot_image(image):
    fix, ax = plt.subplots(figsize=(25, 25))
    ax.imshow(image)
    return ax

In [2]:
raster_path = '../data/city_rasters'
raster_file_name = 'aleppo_2016_10_19_zoom_19.tif'
raster = rasterio.open('{}/{}'.format(raster_path, raster_file_name))
raster_array = RasterSplitter._raster_to_array(raster)

In [3]:
path = '../logs/predictions'
file_name = 'prediction_1560967712.p'
results = pd.read_pickle('{}/{}'.format(path, file_name))
results['prediction_binary'] = (results['prediction'] > 0.5) * 1
print(results.shape)
results.head()

(183652, 2)


prediction  prediction_binary
city   patch_id    date                                     
aleppo 10016-10080 2013-09-23    0.090925                  0
                   2013-10-31    0.028240                  0
                   2014-05-23    0.014604                  0
                   2014-07-14    0.284859                  0
                   2015-10-26    0.210888                  0

In [ ]:
results['prediction_binary'].mean()

In [ ]:
path = '../logs/features'
file_name = 'target_aleppo.p'
target = pd.read_pickle('{}/{}'.format(path, file_name))
#target = target.xs(date(2017, 2, 7), level='date')
print(target.shape)
target.head()

In [ ]:
target.mean()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
ax.hist(results['prediction'], bins=20)
plt.show()

### Real data

In [ ]:
single_date = date(2016, 10, 19)
target_single_date = target.xs(single_date, level='date')
target_destroyed = target_single_date.loc[target_single_date == 1]
target_non_destroyed = target_single_date.loc[target_single_date == 0]
patches_destroyed = pd.DataFrame([elem[1].split('-') for elem in target_destroyed.index.tolist()], columns=['row', 'column']).astype(int)
patches_non_destroyed = pd.DataFrame([elem[1].split('-') for elem in target_non_destroyed.index.tolist()], columns=['row', 'column']).astype(int)

In [ ]:
ax = plot_image(raster_array)
ax.scatter(patches_destroyed['row'], patches_destroyed['column'], color='r', label='destroyed', alpha=0.6, s=1, marker='s')
ax.scatter(patches_non_destroyed['row'], patches_non_destroyed['column'], color='g', label='non-destroyed', alpha=0.6, s=1, marker='s')
plt.legend()
plt.savefig(f'../figures/aleppo_target_{str(single_date)}.pdf', dpi=500)

### Predicted

In [ ]:
results_single_date = results.xs(date(2016, 10, 19), level='date')
prediction_destroyed = results_single_date.loc[results_single_date['prediction_binary'] == 1]
prediction_non_destroyed = results_single_date.loc[results_single_date['prediction_binary'] == 0]
patches_destroyed = pd.DataFrame([elem[1].split('-') for elem in prediction_destroyed.index.tolist()], columns=['row', 'column']).astype(int)
patches_non_destroyed = pd.DataFrame([elem[1].split('-') for elem in prediction_non_destroyed.index.tolist()], columns=['row', 'column']).astype(int)

In [ ]:
ax = plot_image(raster_array)
ax.scatter(patches_destroyed['row'], patches_destroyed['column'], color='r', label='prediction destroyed', alpha=0.8, s=2.2, marker='s')
ax.scatter(patches_non_destroyed['row'], patches_non_destroyed['column'], color='g', label='prediction non-destroyed', alpha=0.8, s=2.2, marker='s')
plt.legend()
plt.savefig(f'../figures/aleppo_predicted_destruction_{str(single_date)}.pdf', dpi=500)

### Correct predictions

In [ ]:
correct_destroyed = target.loc[(target['destroyed'] == 1) & (results['prediction_binary'] == 1)]
correct_non_destroyed = target.loc[(target['destroyed'] == 0) & (results['prediction_binary'] == 0)]
patches_destroyed = pd.DataFrame([elem[1].split('-') for elem in correct_destroyed.index.tolist()], columns=['row', 'column']).astype(int)
patches_non_destroyed = pd.DataFrame([elem[1].split('-') for elem in correct_non_destroyed.index.tolist()], columns=['row', 'column']).astype(int)

In [ ]:
ax = plot_image(raster_array)
ax.scatter(patches_destroyed['row'], patches_destroyed['column'], color='r', label='prediction destroyed', alpha=0.9)
ax.scatter(patches_non_destroyed['row'], patches_non_destroyed['column'], color='g', label='prediction non-destroyed', alpha=0.9)
plt.legend()

### Incorrect predictions

In [ ]:
incorrect_destroyed = target.loc[(target['destroyed'] == 1) & (results['prediction_binary'] == 0)]
incorrect_non_destroyed = target.loc[(target['destroyed'] == 0) & (results['prediction_binary'] == 1)]
patches_destroyed = pd.DataFrame([elem[1].split('-') for elem in incorrect_destroyed.index.tolist()], columns=['row', 'column']).astype(int)
patches_non_destroyed = pd.DataFrame([elem[1].split('-') for elem in incorrect_non_destroyed.index.tolist()], columns=['row', 'column']).astype(int)

In [ ]:
ax = plot_image(raster_array)
ax.scatter(patches_destroyed['row'], patches_destroyed['column'], color='r', label='prediction destroyed', alpha=0.9)
ax.scatter(patches_non_destroyed['row'], patches_non_destroyed['column'], color='g', label='prediction non-destroyed', alpha=0.9)
plt.legend()

## Evolution of destruction over time

In [14]:
plt.ioff()

features_path = '../logs/features/target_aleppo.p'
target = pd.read_pickle(features_path)
for date in target.index.get_level_values('date').unique():
    raster_path = f"../data/city_rasters/aleppo_{str(date.date()).replace('-', '_')}_zoom_19.tif"
    raster = rasterio.open(raster_path)
    raster_array = RasterSplitter._raster_to_array(raster)
    _target = target.xs(date, level='date')
    prediction_destroyed = _target.loc[_target == 1]
    prediction_non_destroyed = _target.loc[_target == 0]
    patches_destroyed = pd.DataFrame([elem[1].split('-') for elem in prediction_destroyed.index.tolist()], columns=['row', 'column']).astype(int)
    patches_non_destroyed = pd.DataFrame([elem[1].split('-') for elem in prediction_non_destroyed.index.tolist()], columns=['row', 'column']).astype(int)
    ax = plot_image(raster_array)
    s = 10000/raster_array.shape[0]
    ax.scatter(patches_destroyed['row'], patches_destroyed['column'], color='r', label='destroyed', alpha=0.8, s=s, marker='s')
    ax.scatter(patches_non_destroyed['row'], patches_non_destroyed['column'], color='g', label='non-destroyed', alpha=0.8, s=s, marker='s')
    plt.legend()
    plt.savefig(f'../figures/target_aleppo_{str(date)}.pdf', dpi=500)

'2013_05_26'

In [ ]:
10000/raster_array.shape[0]

In [ ]:
raster_array.shape[0]/500

## Evolution of predictions over time

In [ ]:
path = '../logs/predictions'
file_name = 'prediction_1559990087.p'
results = pd.read_pickle('{}/{}'.format(path, file_name))
results['prediction_binary'] = (results['prediction'] > 0.5) * 1
for date in results.index.get_level_values('date').unique():
    _results = results.xs(date, level='date')
    prediction_destroyed = _results.loc[_results['prediction_binary'] == 1]
    prediction_non_destroyed = _results.loc[_results['prediction_binary'] == 0]
    patches_destroyed = pd.DataFrame([elem[1].split('-') for elem in prediction_destroyed.index.tolist()], columns=['row', 'column']).astype(int)
    patches_non_destroyed = pd.DataFrame([elem[1].split('-') for elem in prediction_non_destroyed.index.tolist()], columns=['row', 'column']).astype(int)
    ax = plot_image(raster_array)
    ax.scatter(patches_destroyed['row'], patches_destroyed['column'], color='r', label='prediction destroyed', alpha=0.4, s=1, marker='s')
    ax.scatter(patches_non_destroyed['row'], patches_non_destroyed['column'], color='g', label='prediction non-destroyed', alpha=0.4, s=1, marker='s')
    plt.legend()
    plt.savefig(f'prediction_{str(date)}.pdf', dpi=200)